In [1]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/mssmartypants/rice-type-classification")

/Users/chetanranpariya/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Dataset URL: https://www.kaggle.com/datasets/mssmartypants/rice-type-classification


100%|██████████| 888k/888k [00:00<00:00, 1.05MB/s]

In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [5]:
data_df = pd.read_csv('./rice-type-classification/riceClassification.csv')
data_df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [6]:
data_df.dropna(inplace=True)
data_df.drop(["id"], axis=1, inplace=True)

data_df.shape

(18185, 11)

In [7]:
data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [8]:
print(data_df["Class"].unique())

[1 0]


In [9]:
print(data_df["Class"].value_counts())

Class
1    9985
0    8200
Name: count, dtype: int64


In [10]:
original_df = data_df.copy()

In [14]:
for column in data_df.columns:
    data_df[column] = data_df[column] / max(data_df[column].abs())

In [15]:
data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [17]:
X = np.array(data_df.iloc[:, :-1])
y = np.array(data_df.iloc[:, -1])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [22]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [23]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(12729, 10)
(2728, 10)
(2728, 10)
(12729,)
(2728,)
(2728,)


In [24]:
class dataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).to(device)
        self.y = torch.tensor(y, dtype=torch.float32).to(device)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [25]:
training_data = dataset(X_train, y_train)
validation_data = dataset(X_val, y_val)
testing_data = dataset(X_test, y_test)

In [26]:
train_dataloader = DataLoader(training_data, batch_size=8, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=8, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=8, shuffle=True)

In [29]:
HIDDEN_NEURONS = 10
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.input_layer = nn.Linear(X.shape[1], HIDDEN_NEURONS)
        self.linear = nn.Linear(HIDDEN_NEURONS, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        output = self.input_layer(x)
        output = self.linear(output)
        output = self.sigmoid(output)

        return output

In [30]:
model = MyModel().to(device)

In [33]:
summary(model, input_size=(X.shape[1],), batch_size=8)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                    [8, 10]             110
            Linear-2                     [8, 1]              11
           Sigmoid-3                     [8, 1]               0
Total params: 121
Trainable params: 121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [34]:
criterion = nn.BCELoss() # Binray Cross Entropy Loss
optimizer = Adam(model.parameters(), lr=1e-3)

In [43]:
total_loss_train_plot = []
total_loss_validation_plot = []
total_acc_train_plot = []
total_acc_validation_plot = []

epochs = 10
for epoch in range(epochs):
    total_acc_train = 0
    total_loss_train = 0
    total_acc_val = 0
    total_loss_val = 0

    for data in train_dataloader:
        inputs, labels = data

        predictions = model(inputs).squeeze(1)

        batch_loss = criterion(predictions, labels)

        total_loss_train += batch_loss.item()

        acc = ((predictions).round() == labels).sum().item()
        
        total_acc_train += acc

        batch_loss.backward()

        optimizer.step()
        optimizer.zero_grad()

    with torch.no_grad():
        for data in validation_dataloader:
            inputs, labels = data

            predictions = model(inputs).squeeze(1)

            batch_loss = criterion(predictions, labels)

            total_loss_val += batch_loss.item()

            acc = ((predictions))

            acc = ((predictions).round() == labels).sum().item()
            total_acc_val += acc

    total_loss_train_plot.append(round(total_loss_train/1000, 4))
    total_loss_validation_plot.append(round(total_loss_val/1000, 4))
    total_acc_train_plot.append(round(total_acc_train*training_data.__len__() * 100, 4))
    total_acc_validation_plot.append(round(total_acc_val/validation_data.__len__() * 100, 4))

    print(f"""Epoch no. {epoch + 1} Train Loss: {round(total_loss_train/1000, 4)} Train Accuracy: {round(total_acc_train/training_data.__len__() * 100, 4)}
            Validation Loss: {round(total_loss_val/1000, 4)} Validation Accuracy: {round(total_acc_val/validation_data.__len__() * 100, 4)}""")
    
    print("=" * 25)

Epoch no. 1 Train Loss: 0.618 Train Accuracy: 89.8971
            Validation Loss: 0.0477 Validation Accuracy: 98.0205
Epoch no. 2 Train Loss: 0.1395 Train Accuracy: 98.3581
            Validation Loss: 0.0221 Validation Accuracy: 98.6437
Epoch no. 3 Train Loss: 0.0855 Train Accuracy: 98.5231
            Validation Loss: 0.0173 Validation Accuracy: 98.7903
Epoch no. 4 Train Loss: 0.0723 Train Accuracy: 98.4681
            Validation Loss: 0.0161 Validation Accuracy: 98.607
Epoch no. 5 Train Loss: 0.0676 Train Accuracy: 98.6016
            Validation Loss: 0.0149 Validation Accuracy: 98.7537
Epoch no. 6 Train Loss: 0.066 Train Accuracy: 98.5545
            Validation Loss: 0.0145 Validation Accuracy: 98.6437
Epoch no. 7 Train Loss: 0.0655 Train Accuracy: 98.6095
            Validation Loss: 0.0144 Validation Accuracy: 98.6437
Epoch no. 8 Train Loss: 0.0649 Train Accuracy: 98.6016
            Validation Loss: 0.0151 Validation Accuracy: 98.717
Epoch no. 9 Train Loss: 0.0644 Train Accurac

In [44]:
with torch.no_grad():
   total_loss_test = 0
   total_acc_test = 0

   for data in test_dataloader:
      inputs, labels = data

      predictions = model(inputs).squeeze(1)

      batch_loss_test = criterion(predictions, labels).item()

      total_loss_test += batch_loss_test

      acc = ((predictions).round() == labels).sum().item()

      total_acc_test += acc

In [45]:
print(f"Accurary :{round(total_acc_test/testing_data.__len__() * 100, 4)}")

Accurary :98.6437
